# Régression

## Objectif

Dans cette partie "Régression", on cherche à prédire les émissions de CO2 de voitures, en se basant sur leurs caractéristiques \(poids, carburant utilisé, etc\). L'objectif est de minimiser la moyenne des erreurs absolues entre les valeurs prédites et les valeurs réelles.

## Traitement des données

### Importation des dépendances

Nous importons les dépendances python en début de fichier pour faciliter leur installation.

# Regréssion avec Scikit learn

Dans cette partie, nous allons reprendre notre reégrression linéaire mais avec la bibliothèque Scikit-learn, qui propose des modèles très interessants

In [180]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.linear_model import RidgeCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import pandas as pd
import sklearn
import numpy as np

## Importation des données

In [216]:
df_train = pd.read_csv("dataset/train.csv")
df_test = pd.read_csv("dataset/test.csv")

On supprime la colonne "id". On sépare la cible des autres variables

In [217]:
target_name = "co2"
target, data = df_train[target_name] , df_train.drop(columns=[target_name,"id"])   #On sépare la colonne target des autres colonnes

## Vérifions les valeurs nulles

Beaucoup de valeurs nulles dans les colonnes hc, nox, hcnox

In [6]:
data.isna().sum()

brand                   0
model                   0
car_class               0
range                   0
fuel_type               0
hybrid                  0
max_power               0
grbx_type_ratios        0
weight_min              0
weight_max              0
urb_cons                6
exturb_cons             6
overall_cons            0
co                     86
hc                  33890
nox                    86
hcnox                7416
ptcl                 1965
dtype: int64

## Parcours des données

In [10]:
data.head()

,brand,model,car_class,range,fuel_type,hybrid,max_power,grbx_type_ratios,weight_min,weight_max,urb_cons,exturb_cons,overall_cons,co,hc,nox,hcnox,ptcl
0,MERCEDES,COMBI 110 CDI,MINIBUS,MOY-INFER,GO,non,70.0,M 6,1976,2075,9.1,6.4,7.4,0.083,NaN,0.229,0.250,0.001
1,MERCEDES,VIANO 2.0 CDI,MINIBUS,MOY-SUPER,GO,non,100.0,A 5,2186,2355,10.2,7.0,8.2,0.078,NaN,0.224,0.233,0.001
2,MERCEDES,SPRINTER COMBI 319 CDI,MINIBUS,MOY-INFER,GO,non,140.0,A 5,2586,2869,12.5,9.0,10.3,0.067,0.014,1.846,NaN,0.002
3,RENAULT,MEGANE Coupé EnergyTCe (115ch) eco2,COUPE,MOY-INFER,ES,non,85.0,M 6,1280,1280,6.4,4.6,5.3,0.167,0.039,0.039,NaN,0.001
4,MERCEDES,COMBI 116 CDI,MINIBUS,MOY-INFER,GO,non,120.0,A 5,2356,2450,10.1,6.9,8.1,0.042,NaN,0.190,0.201,0.001


In [11]:
data.describe()

,max_power,weight_min,weight_max,urb_cons,exturb_cons,overall_cons,co,hc,nox,hcnox,ptcl
count,41257.000000,41257.000000,41257.000000,41251.000000,41251.000000,41257.000000,41171.000000,7367.000000,41171.000000,33841.000000,39292.000000
mean,118.982960,2102.832901,2342.408609,9.570216,6.728448,7.761779,0.181025,0.026455,0.300336,0.233684,0.000896
std,45.282568,293.810948,423.303792,2.079990,1.035866,1.378721,0.145250,0.019502,0.418976,0.037348,0.000994
min,28.000000,825.000000,825.000000,0.000000,2.800000,0.600000,0.005000,0.000000,0.000000,0.000000,0.000000
25%,100.000000,1982.000000,2075.000000,8.800000,6.500000,7.300000,0.061000,0.008000,0.197000,0.216000,0.000000
50%,120.000000,2076.000000,2355.000000,9.400000,6.900000,7.800000,0.137000,0.029000,0.214000,0.239000,0.001000
75%,120.000000,2246.000000,2709.000000,10.200000,7.200000,8.300000,0.297000,0.042000,0.228000,0.253000,0.001000
max,585.000000,2760.000000,3094.000000,41.099998,14.900000,24.500000,0.968000,0.510000,1.846000,0.570000,0.023000


## Transformation des Données

----- D'après la description des données hcnox = hc +nox

Ainsi, on peut donc retrouver les valeurs nulles dans la colonne hc à partir de hcnox et nox. Une fois cela fait, on supprime la colonne hcnox.

In [218]:
# Remplir hc à partir de hcnox et nox
is_empty_hc = (data['hc'].isnull() & data['hcnox'].notnull() & data['nox'].notnull())
data.loc[is_empty_hc,'hc'] = data.loc[is_empty_hc,'hcnox'] - data.loc[is_empty_hc,'nox']

#Remplir hcnox à partir de hc et nox
is_empty_hcnox = (data['hcnox'].isnull() & data['hc'].notnull() & data['nox'].notnull())
data.loc[is_empty_hcnox,'hcnox'] = data.loc[is_empty_hcnox,'hc'] + data.loc[is_empty_hcnox,'nox']

---- Variable Range

Après inspection des données, on considère que MOY-INFER = MOY-INFERIEURE. On peut supposer une erreur de saisie

In [192]:
data["range"].value_counts()

range
MOY-INFER     25156
MOY-SUPER     11275
LUXE           2677
SUPERIEURE     1163
INFERIEURE      800
ECONOMIQUE      186
Name: count, dtype: int64

In [164]:
is_moy_infer = (data["range"]=="MOY-INFERIEURE")
data.loc[is_moy_infer, "range"] = "MOY-INFER"

### --- Model
Trop de valeurs différentes(3141). Il serait judicieux de la supprimer

In [15]:
print(data["model"].value_counts())
print(data["model"].nunique())

model
VIANO 2.2 CDI                                                  4405
VIANO 2.0 CDI                                                  2931
COMBI 116 CDI                                                  2847
COMBI 113 CDI                                                  1968
VIANO 3.0 CDI                                                  1180
                                                               ... 
2008 1.6 e-Hdi FAP BVM6                                           1
A6 2.0 TDI (136ch) MULTITRONIC 8                                  1
PASSAT SW 1.6 TDI (105ch) CR FAP BMT BVM6                         1
MULTIVAN CONFORTLINE ET HIGHLINE 2.0 TDI (180ch) BlueMotion       1
ASX CLEARTEC 1.8 DI-D Invite / Intense / Instyle 4WD              1
Name: count, Length: 3141, dtype: int64
3141


Si on conserve la vairable "model", notre modèle de ML est moins performant (0.1164 contre 0.0718 sans la variable)

In [161]:
#model_counts = data["model"].value_counts()
#models_to_replace = model_counts[model_counts < 500].index

# Remplacer les modèles peu courants par "AUTRE"
#data.loc[data["model"].isin(models_to_replace), "model"] = "AUTRE"

On supprime alors la varaible du modèle

In [219]:
data = data.drop(columns="model")

### --- Car Class

Erreur de nommeclature. COMBISPACACE = COMBISPACE. Un 'C' en trop s'est peut etre introduit lors de la saisie

In [221]:
print(data["car_class"].value_counts())
print(data["car_class"].nunique())

car_class
MINIBUS                34570
BERLINE                 3307
BREAK                    933
TS TERRAINS/CHEMINS      819
COUPE                    661
CABRIOLET                389
MONOSPACE COMPACT        240
COMBISPACE               183
MINISPACE                111
MONOSPACE                 44
Name: count, dtype: int64
10


In [166]:
is_moy_combispace = (data["car_class"]=="COMBISPCACE")
data.loc[is_moy_combispace, "car_class"] = "COMBISPACE"

### ---- Variable grbx_type_ratios

Scinder la colonnne "grbx_type_ratios" en deux colonnes :  le Gear_box type et le nombre de ratio

In [48]:
data["grbx_type_ratios"].value_counts()

grbx_type_ratios
M 6    23823
A 5    10169
A 7     5103
M 5      738
A 6      647
A 8      385
V 0      238
D 5       51
A 9       27
D 6       23
A 4       19
D 7       18
M 7       13
V .        2
S 6        1
Name: count, dtype: int64

In [220]:

data[['gearbox', 'ratio']] = data["grbx_type_ratios"].str.split(' ', n=1, expand=True)
data['ratio'] = data['ratio'].replace({'.': '0'})
data = data.drop(columns="grbx_type_ratios")

----- Variable Brand

Beaucoup de valeurs différentes (44). On peut conserver les deux premières car elles sont les plus présentes dans le dataset

In [50]:
print(data["brand"].value_counts())
print(f"Nombre de valeurs différentes dans la colonne Brand {data['brand'].nunique()}")

brand
MERCEDES        27127
VOLKSWAGEN      10266
FIAT              394
BMW               342
OPEL              308
LEXUS             277
AUDI              264
FORD              220
NISSAN            215
CITROEN           166
ALFA-ROMEO        142
PEUGEOT           125
RENAULT           109
TOYOTA            103
SKODA              96
MINI               95
VOLVO              86
PORSCHE            82
SEAT               78
CHEVROLET          61
JEEP               60
LAND ROVER         60
ASTON MARTIN       55
KIA                53
HONDA              43
HYUNDAI            43
LANCIA             41
MITSUBISHI         35
MAZDA              31
CADILLAC           31
JAGUAR             29
SMART              29
LAMBORGHINI        26
DACIA              22
SUZUKI             21
SUBARU             19
LOTUS              18
FERRARI            18
MASERATI           16
ROLLS-ROYCE        15
BENTLEY            14
INFINITI           12
SSANGYONG           6
LADA                4
Name: count, dtype: int64


In [221]:
is_not_abundant = ~data["brand"].isin(["MERCEDES", "VOLKSWAGEN"])
data.loc[is_not_abundant, "brand"] = "AUTRE"

In [125]:
print(data["brand"].value_counts())

brand
MERCEDES      27127
VOLKSWAGEN    10266
AUTRE          3864
Name: count, dtype: int64


### ---- Fuel Type

Nous avons d'abord voulu fusionner GN/ES et ES/GN. Mais après des recherches, nous avons vu que c'est deux etiquetes differentes

In [226]:
print(data["fuel_type"].value_counts())
print(data["fuel_type"].nunique())

fuel_type
GO       36993
ES        3853
AUTRE      411
Name: count, dtype: int64
3


### Utilisons Scikit learn pour diviser nos données

In [238]:
x_train, x_val, y_train, y_val = train_test_split(data, target, test_size=0.2, random_state=1)
x_train = x_train.reset_index().drop('index', axis=1)
x_val = x_val.reset_index().drop('index', axis=1)

In [55]:
x_train.head()

,brand,car_class,range,fuel_type,hybrid,max_power,weight_min,weight_max,urb_cons,exturb_cons,overall_cons,co,hc,nox,ptcl,gearbox,ratio
0,MERCEDES,MINIBUS,MOY-INFER,GO,non,95.0,2076,2185,10.0,6.6,7.9,0.483,0.066,0.221,0.000,M,6
1,MERCEDES,MINIBUS,MOY-INFER,GO,non,95.0,2356,2585,10.1,7.4,8.4,0.100,0.006,0.247,0.001,A,7
2,VOLKSWAGEN,MINIBUS,MOY-INFER,GO,non,120.0,2307,2815,8.0,6.6,7.1,0.401,0.038,0.213,0.001,M,6
3,MERCEDES,MINIBUS,MOY-SUPER,GO,non,100.0,2075,2075,8.8,6.4,7.2,0.175,0.022,0.226,0.001,M,6
4,MERCEDES,MINIBUS,MOY-INFER,GO,non,120.0,2186,2355,8.9,6.5,7.3,0.046,0.019,0.197,0.000,M,6


## Sélection des variables numériques et catégoriques

In [222]:
numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(data)
categorical_columns = categorical_columns_selector(data)

## Pipeline de préprocessing

In [223]:
categorical_preprocessor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))])

# Define the preprocessing steps for numeric variables
numerical_preprocessor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])


#Notre modèle qui est un pipeline, standarisation des données puis, regréssion logistique
preprocessors = ColumnTransformer([
    ('categorical preprocessing', categorical_preprocessor, categorical_columns),
    ('numerical preprocessing', numerical_preprocessor, numerical_columns)])

## Construction des modèles

Faisons d'abord les fonctions de prédictions et de Cross Validation

### Fonction pour le prétraitement des données de test

In [224]:
def preprocess(df_test):
    # Remplir hc à partir de hcnox et nox
    is_empty_hc = (df_test['hc'].isnull() & df_test['hcnox'].notnull() & df_test['nox'].notnull())
    df_test.loc[is_empty_hc,'hc'] = df_test.loc[is_empty_hc,'hcnox'] - df_test.loc[is_empty_hc,'nox']

    #Remplir hcnox à partir de hc et nox
    is_empty_hcnox = (df_test['hcnox'].isnull() & df_test['hc'].notnull() & df_test['nox'].notnull())
    df_test.loc[is_empty_hcnox,'hcnox'] = df_test.loc[is_empty_hcnox,'hc'] + df_test.loc[is_empty_hcnox,'nox']
    df_test = df_test.drop(columns=["id","model"])

    is_moy_combispace = (df_test["car_class"]=="COMBISPCACE")
    df_test.loc[is_moy_combispace, "car_class"] = "COMBISPACE"

    is_moy_infer = (df_test["range"]=="MOY-INFERIEURE")
    df_test.loc[is_moy_infer, "range"] = "MOY-INFER"

    df_test[['gearbox', 'ratio']] = df_test["grbx_type_ratios"].str.split(' ', n=1, expand=True)
    df_test['ratio'] = df_test['ratio'].replace({'.': '0'})                                          #Remplacer les points par zéro
    df_test = df_test.drop(columns="grbx_type_ratios")

    is_not_abundant = ~df_test["brand"].isin(["MERCEDES", "VOLKSWAGEN"])
    df_test.loc[is_not_abundant, "brand"] = "AUTRE"

    return df_test

### Fonction pour avoir les prédictions

In [91]:
def get_prediction(X_val, y_val, model):
    pred = model.predict(X_val)
    
    # Calculate MAE
    mae = mean_absolute_error(y_val, pred)

    print("Mean Absolute Error:" , mae)

### Fonction pour la Cross Validation

In [257]:
def validation(model,data,target,nb_validate):    
    cv_results = cross_validate(
        model, data, target, cv=nb_validate, return_estimator=True,           #nb_validate validations croisées
        return_train_score=True, scoring="neg_mean_squared_error"
    )
    scores_train=-cv_results["train_score"]
    scores_test=-cv_results["test_score"]
    print(f"Erreur quadratique moyenne du modèle de régression linéaire sur les données d'entrainement:\n"
        f"{scores_train.mean():.3f} ± {scores_train.std():.3f}")

    print(f"Erreur quadratique moyenne du modèle de régression linéaire sur les données de test:\n"
        f"{scores_test.mean():.3f} ± {scores_test.std():.3f}")    

### Fonction de submit

In [129]:
def submit(model,df_test):
    df_test_processed = preprocess(df_test)
    predictions  = model.predict(df_test_processed)
    submit = pd.DataFrame({'id': df_test["id"], 'co2': predictions})
    submit.to_csv("predictions.csv", index=False)

## -- XGBoost Pipeline (Notre meilleur modèle : 0.0704)

In [225]:
xg_regressor = XGBRegressor(n_estimators=15000, 
                     random_state=0, 
                     learning_rate=0.01,
                     max_depth=200,
                     enable_categorical=True)

xg_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessors),
        ('regressor', xg_regressor)
    ])

In [ ]:
xg_pipeline.fit(x_train,y_train)

In [ ]:
get_prediction(x_val, y_val, xg_pipeline)

### Hyper parameter Tuning du XGBoost

In [120]:
param_grid = {
    'regressor__n_estimators': [10000,15000,20000],
    'regressor__max_depth': [100,200,500,1000],
    'regressor__learning_rate': [0.01, 0.05, 0.1,0.15]
}

RandomizedSearchCV a été préféré à GridSearchCV car GridSearchCV teste toutes les combinaisons ce qui prend énormément de temps.

In [121]:
random_cv = RandomizedSearchCV(
    estimator=xg_pipeline,
    param_distributions=param_grid,
    cv=3,                                           #Trois validations croisées pour déterminer les meilleures hyper paramètres
    n_iter=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=5,
    return_train_score=True,
    random_state=0
)

In [122]:
random_cv.fit(x_train, y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('categorical '
                                                                               'preprocessing',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(fill_value='missing',
                                                                                                              strategy='constant')),
                                                                                               ('encoder',
                                                                                                OneHotEncoder(handle_unknown='ignore'))]),
                                                                               ['brand',
                                                                                'car_class',
                                                                                'range',
                                                                                'fuel_type',
                                                                                'hybrid',
                                                                                'gearbox']),
                                                                              ('numerical '
                                                                               'preprocess...
                                                           multi_strategy=None,
                                                           n_estimators=10000,
                                                           n_jobs=None,
                                                           num_parallel_tree=None,
                                                           random_state=0, ...))]),
                   n_iter=5, n_jobs=-1,
                   param_distributions={'regressor__learning_rate': [0.01, 0.05,
                                                                     0.1,
                                                                     0.15],
                                        'regressor__max_depth': [100, 200, 500,
                                                                 1000],
                                        'regressor__n_estimators': [10000,
                                                                    15000,
                                                                    20000]},
                   random_state=0, return_train_score=True,
                   scoring='neg_root_mean_squared_error', verbose=5)

On a nos meilleurs hyperparamètres. Ceux-ci ont directement été remplacé dans le code d'en haut 

In [123]:
random_cv.best_params_

{'regressor__n_estimators': 15000,
 'regressor__max_depth': 200,
 'regressor__learning_rate': 0.01}

### Testons notre modèle et création du fichier pour le submit sur Kaggle

In [228]:
xg_pipeline.fit(data,target)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('categorical preprocessing',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['brand', 'car_class',
                                                   'range', 'fuel_type',
                                                   'hybrid', 'gearbox',
                                                   'ratio']),
                                                 ('numerical preprocessing',
                                                  Pipeline(steps=[('imputer...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.001,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=200, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=15000, n_jobs=None,
                              num_parallel_tree=None, random_state=0, ...))])

In [229]:
submit(xg_pipeline,df_test)